In [4]:
%%sh
pwd

/home/wce/clsadmin


In [ ]:
%%sh
pip install python-dotenv

In [ ]:
%%sh
pip install plotly
pip install pyspark
pip install geopandas==0.3.0
pip install pyshp==1.2.10
pip install shapely==1.6.3

In [ ]:
# Import packages
import plotly as plt
import pandas as pd
import dotenv as dt
from pyspark.sql import SparkSession

In [ ]:
%%sh
pip freeze

In [ ]:
%%sh
ls data

In [ ]:
# reading in data
df = spark.read.csv("/data/ca/*", header=True)

In [ ]:
df.head(10)

In [ ]:
df.select("LON").show()

In [ ]:
# creating a dataframe of just longitudes and latitudes, potentially for future use
lonlat = df.select(df["LAT"], df["LON"])

In [ ]:
# this his how we need to feed the 
import requests
response = requests.get("https://geo.fcc.gov/api/census/block/find?latitude=33.786405&longitude=-86.9329571&format=xml")

In [ ]:
# json format is returned, but successful request to api
print(response.content)

In [ ]:
# simple proof of concept to show that we cna feed LON and LAT into the below
LON = -118.219827
LAT = 34.0424273


test = requests.get("https://geo.fcc.gov/api/census/block/find?latitude=%s&longitude=%s&format=xml" % (LAT,LON))
print(test.content)

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import udf, struct

#df_test = pd.DataFrame(df.head(10), columns=df.columns)

get_fips = udf(lambda x, y: 
                   requests.get("https://geo.fcc.gov/api/census/block/find?latitude=%s&longitude=%s&format=xml" % (str(x),str(y))), IntegerType())



#k = df.withColumn("FIPS",get_fips(df["LAT"],df["LON"] ))

In [ ]:
get_fips = udf(lambda x: "https://geo.fcc.gov/api/census/block/find?latitude=%s&longitude=%s&format=xml" % (x.LAT,x.LON), IntegerType())

In [ ]:
k = df.rdd.map(lambda x: requests.get("https://geo.fcc.gov/api/census/block/find?latitude=%s&longitude=%s&format=xml" % (x.LAT,x.LON)))

In [ ]:
test_df = df.withColumn("FIPS",get_fips(df["LAT"], df["LON"))

In [ ]:
from pyspark.sql.functions import udf

# Use udf to define a row-at-a-time udf
@udf('double')
# Input/output are both a single double value
def plus_one(v):
      return v + 1

df.withColumn('LONPLUS', plus_one(df.LON))

In [ ]:
df.select('LONPLUS')